Created on Monday 04 January 2021

Group 5 - Classification

@authors : FB


In [ ]:
!pip install langdetect # To detect languages
!pip install googletrans # l'API google traduction, au cas besoin

     |████████████████████████████████| 983kB 5.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=40d5ea7208ba321f84d5739d99f88f28e13de9d189db683bca0fcbea06da7489
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import plotly.express as px
from langdetect import detect

import os
os.chdir("/content/drive/My Drive/G5 Inter-Promo 2021/Données/Input")

In [ ]:
# Read data
data=pd.read_json('Data.json')
data.index=data['art_id']
data.drop('art_id', axis=1, inplace=True)
data.head()

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
art_id,,,,,,,,,,,,
0,,None,7 décembre 2018,fr,4ème Conférence Nationale de l’Emploi Territor...,http://fncdg.com/4eme-conference-nationale-de-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2018/12/ra...,None,None
1,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None
2,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None
3,,None,28 juin 2017,fr,Arrêté de création GIP,http://fncdg.com/arrete-de-creation-gip/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/fo...,None,None
4,,None,24 novembre 2017,fr,Article Acteurs Publics « Les centres de gesti...,http://fncdg.com/article-acteurs-publics-les-c...,FNCDG,xpath_source,http://fncdg.com/actualites/,None,None,None


# Description

In [ ]:
# The different types of languages
pd.unique(data.art_lang.values)

array(['fr', 'en', None], dtype=object)

In [ ]:
# Put the None in other
art_id=data[data['art_lang'].isnull()].index
data.loc[art_id,'art_lang']='other'

In [ ]:
def camembert(col:str)->print:
  '''A pie chart display function.
     You give it the column's name and display the different values in pie chart.
     col: column's name
  '''
  data_copy = data[col].value_counts().reset_index()
  data_copy.columns = [col, 'count']
  fig = px.pie(data_copy, values = 'count', names = col)
  fig.show()

In [ ]:
camembert('art_lang')

In [ ]:
# Detecting the language of other content

art_id_autre=data[data['art_lang']=="other"].index
for art_id in art_id_autre:
  content=data.loc[art_id,'art_content']
  lang=detect(content)
  data.loc[art_id, 'art_lang']=lang

camembert('art_lang')

In [ ]:
# The different values of the types of sources
pd.unique(data.src_type.values)

array(['xpath_source', 'rss_source', 'keyword_source'], dtype=object)

In [ ]:
# Camembert from different sources
camembert('src_type')

In [ ]:
len(pd.unique(data.src_name.values))

585

In [ ]:
len(pd.unique(data.art_auth.values))

673

# Cleaning

In [ ]:
!python3 -m spacy download fr_core_news_md

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('french') 

from string import punctuation

import re
import collections as clt
import spacy
import fr_core_news_md
nlp = fr_core_news_md.load()


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_fr=data[data['art_lang']=='fr'] # Work only with data in French

In [ ]:
data_fr.sample(15).art_content.values.tolist() #Brief overview for cleaning

["04/12/2020 Jusqu’à fin 2020, le Grand Chalon augmente de 50% les aides des dispositifs Renov + et Adapt +. De plus, le public non éligible aux aides de l’Anah est concerné. Parmi les objectifs : soutenir les entreprises locales du bâtiment. Fin 2017, le Grand Chalon (51 communes, 118 000 habitants) a mis en place un régime spécifique d’aides à la rénovation des logements privés. Avec à la clé deux dispositifs financiers, l’un pour aider les propriétaires de maisons individuelles aux travaux de rénovation énergétique (Renov +), et l’autre, pour l’adaptation des logements anciens à une diminution de l’autonomie du propriétaire (Adapt +). Des aides incitatives au passage à l’acte car, pour Renov +, elles sont cumulables avec celles de l’Anah pour ces types de travaux. Dans le contexte de crise sanitaire et de son impact économique et social, la communauté d’agglomération a décidé de passer à la vitesse supérieure. Depuis le 1er juin et jusqu'au 31 décembre, elle majore ainsi de 50% le m

In [ ]:
# A completer en groupe

Email =re.compile('[a-zA-Z0-9\.-]+@[a-zA-Z\._/-]+\.[a-z]+')
Site =re.compile('(http://)?(www\.)?[a-z/\.]+\.[a-z/]+')
Chiffre=re.compile('[0-9/%\.,-]+')
Caractere=re.compile('\W+')


In [ ]:

def clean(content:str)->str:
  ''' A function that cleans the contents
      content : content
  '''


  # A completer en groupe
  content=Email.sub(' ',content)
  content=Site.sub(' ',content)
  content=Chiffre.sub(' ',content)
  content=Caractere.sub(' ',content)
  content=content.lower()

  for p in punctuation:
    content=content.replace(p," ")

  content_=nlp(content)
  content_lemma=[]
  for token in content_:
    if str(token) not in stop_words:
      content_lemma.append(token.lemma_)
  
  return ' '.join(content_lemma)

In [ ]:
%%time
data_fr['art_content_clean']=data_fr['art_content'].apply(clean)

CPU times: user 28min 43s, sys: 1min 38s, total: 30min 22s
Wall time: 30min 24s


In [ ]:
content_sup=['error',''] # Remove contents that contain only error 404 and are empty.
for cs in content_sup:
  data_fr=data_fr[data_fr['art_content_clean']!=cs]

# TF_IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# min_df et/ou  max_df à définir en groupe

vectorizer = TfidfVectorizer(min_df=0.1)
X = vectorizer.fit_transform(data_fr['art_content_clean'].values)
vocabulaire=vectorizer.get_feature_names()

In [ ]:
X.shape, len(vocabulaire)

((11598, 629), 629)

In [ ]:
tf_idf=pd.DataFrame(X.toarray(), columns=vocabulaire)
tf_idf.head()

,abord,accompagnement,accompagner,accès,achat,acteur,action,activité,actuel,adapter,adcf,administratif,administration,adopter,affaire,afin,agence,agent,agir,aide,aider,ailleurs,ainsi,ajouter,aller,alors,amélioration,améliorer,aménagement,an,analyse,annoncer,année,appel,appeler,application,apporter,appuyer,après,article,...,urbain,urgence,usage,utilisation,utiliser,valeur,venir,vers,via,vie,ville,viser,vision,voir,vouloir,vue,véritable,web,échange,échanger,échelle,écologique,économie,économique,également,élu,élément,énergie,équipe,établir,établissement,étape,état,étude,éviter,évoluer,évolution,évoquer,être,œuvre
0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.00000,0.000000
1,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.335685,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.079172,0.0,0.119826,0.0,0.000000,0.0,0.000000,0.067102,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.119798,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.117149,0.0,0.0,0.0,0.120136,0.0000,0.0,0.000000,0.00000,0.000000
2,0.0,0.0,0.021489,0.048719,0.0,0.021896,0.018242,0.000000,0.0,0.0,0.00000,0.027748,0.000000,0.026172,0.0,0.018877,0.0,0.285710,0.0,0.024062,0.0,0.022203,0.063999,0.0,0.072518,0.0,0.0,0.024572,0.0,0.0,0.0,0.0,0.016443,0.0,0.0,0.022638,0.022377,0.0,0.021035,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.019306,0.046527,0.0,0.02416,0.0,0.0,0.0,0.020881,0.0,0.025756,0.0,0.0,0.024461,0.000000,0.0,0.0,0.0,0.000000,0.031370,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0292,0.0,0.028166,0.01458,0.044965
3,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.111554,0.000000,0.0,0.0,0.00000,0.000000,0.166844,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.195680,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.172788,0.0,0.0,0.0,0.000000,0.000000,0.148194,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.00000,0.000000
4,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.10727,0.000000,0.116757,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.136936,0.0,0.000000,0.0,0.0,0.105151,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.199107,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.081639,0.067122,0.103706,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.00000,0.000000


In [ ]:
#tf_idf.to_csv("../Output/g5_FB_tf_idf_v0.csv", index=False)